In [2]:
import json
from eval.eval_molund import tranform_str_to_json, check_string_type
from eval.eval_molund import eval_molund_from_list
print("success")

success


In [4]:
def evaluate_molund_score(model_name):
    task_dict = dict(
        fg_samples="fg_samples", murcko='murcko_scaffold', ring_count='ring_count',
        ring_system='ring_system_scaffold', mutated='mutated', permutated='permutated'
    )
    pred_key_dict = dict(
        fg_samples="count", murcko='Output Scaffold', ring_count='count',
        ring_system='output', mutated='output', permutated='output'
    )
    gt_key_dict = dict(
        fg_samples="fg_num", murcko='largest_scaffold', ring_count='count',
        ring_system='', mutated='', permutated=''
    )
    result_dict = dict()
    
    for task in task_dict.keys():
        print(model_name, task)
        if 'llama' not in model_name:
            file_name = f"logs/{task_dict[task]}/{model_name}.json"
            
        pred_results = json.load(open(file_name, "r"))
        invalid_number = 0
        
        pred_list, gt_list = list(), list()
        for pred in pred_results:  
            if type(pred['json_results']) is str:
                pred_json = tranform_str_to_json(str_input=pred['json_results'])
                # if model_name == 'gemini': pred_json = pred_json[0]
                if pred_json == None:
                    invalid_number += 1
                    continue
                else:
                    if pred_key_dict[task] not in pred_json.keys():
                        invalid_number += 1; continue
                    if pred_json[pred_key_dict[task]] == "": 
                        invalid_number += 1; continue
                    if task in ["ring_count", "fg_samples"]:
                        if check_string_type(pred_json[pred_key_dict[task]]) == "string":
                            invalid_number += 1; continue;
                    pred_list.append(pred_json[pred_key_dict[task]])
                    if gt_key_dict[task] != "":
                        gt_list.append(pred[gt_key_dict[task]])
            else:
                if pred_key_dict[task] not in pred['json_results'].keys():
                    invalid_number += 1; continue
                if pred['json_results'][pred_key_dict[task]] == "": 
                    invalid_number += 1; continue
                if task in ["ring_count", "fg_samples"]:
                    if check_string_type(pred['json_results'][pred_key_dict[task]]) == "string":
                            invalid_number += 1; continue;
                pred_list.append(pred['json_results'][pred_key_dict[task]])
                if gt_key_dict[task] != "":
                    gt_list.append(pred[gt_key_dict[task]])
        
        assert len(pred_results) == invalid_number+len(pred_list)
        result_dict[task] = eval_molund_from_list(gt_list=gt_list, pred_list=pred_list, total_number=len(pred_results), task=task)
        print(model_name, task, result_dict[task])
    
    print(f"eval_score_{model_name}", result_dict)
    # json.dump(result_dict, open(f"logs/eval_score_{model_name}.json", "w"), indent=4)


if __name__ == "__main__":
    model_list = ['claude3.7']
    for model_name in model_list:
        evaluate_molund_score(model_name=model_name)

claude3.7 fg_samples
claude3.7 fg_samples {'score': 0.21, 'fg_samples-valid-rate': 1.0}
claude3.7 murcko
claude3.7 murcko {'score': 0.3995904332070048, 'murcko-valid-rate': 0.95}
claude3.7 ring_count
claude3.7 ring_count {'score': 1.6, 'ring_count-valid-rate': 1.0}
claude3.7 ring_system
claude3.7 ring_system {'score': 0.7948717948717948, 'ring_system-valid-rate': 0.975}
claude3.7 mutated
claude3.7 mutated {'score': 0.7209302325581395, 'mutated-valid-rate': 0.86}
claude3.7 permutated
claude3.7 permutated {'score': 0.9591836734693877, 'permutated-valid-rate': 0.98}
eval_score_claude3.7 {'fg_samples': {'score': 0.21, 'fg_samples-valid-rate': 1.0}, 'murcko': {'score': 0.3995904332070048, 'murcko-valid-rate': 0.95}, 'ring_count': {'score': 1.6, 'ring_count-valid-rate': 1.0}, 'ring_system': {'score': 0.7948717948717948, 'ring_system-valid-rate': 0.975}, 'mutated': {'score': 0.7209302325581395, 'mutated-valid-rate': 0.86}, 'permutated': {'score': 0.9591836734693877, 'permutated-valid-rate': 0